In [ ]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy
from pennylane import BasisState, FermionicSingleExcitation, FermionicDoubleExcitation
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Molecular Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(
        symbols, geometry, basis="sto-3g", charge=charge, method="pyscf"
    )

    # Hartree-Fock reference
    hf_state = qml.qchem.hf_state(electrons, qubits)

    # Single and double excitations
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    pool = singles + doubles
    wires = list(range(qubits))
    params = np.zeros(len(pool))
    qml.BitFlip(p=0.1, wires=0)
    # Device
    dev = qml.device("default.mixed", wires=qubits, shots=shots)

    if shots is None:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            # Initialize HF state
            BasisState(hf_state, wires=wires)
            
            # Ensure params shape compatible
            param_array = params
            if len(params.shape) == 1:
                param_array = np.expand_dims(params, 0)

            # Apply UCCSD excitations
            for layer in range(1):
                # Double excitations
                for i, (w1, w2) in enumerate(d_wires):
                    FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
                # Single excitations
                for j, s_w in enumerate(s_wires):
                    FermionicSingleExcitation(param_array[layer][j], wires=s_w)

            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            BasisState(hf_state, wires=wires)
            qml.BitFlip(p=0.1, wires=1)
            param_array = params
            if len(params.shape) == 1:
                param_array = np.expand_dims(params, 0)

            for layer in range(1):
                for i, (w1, w2) in enumerate(d_wires):
                    FermionicDoubleExcitation(param_array[layer][len(s_wires)+i], wires1=w1, wires2=w2)
                for j, s_w in enumerate(s_wires):
                    FermionicSingleExcitation(param_array[layer][j], wires=s_w)

            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(
            circuit, params,
            wires=wires, s_wires=s_wires,
            d_wires=d_wires, hf_state=hf_state
        )

    gr_state = circuit(params, wires, s_wires, d_wires, hf_state)
    print(f"Ground state energy: {gr_state:.8f} Ha")
    return params, pool

r = 1.88973
symbols = ['H', 'H', 'H', 'H']
#symbols = ['H', 'H']
geometry = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 1.5*r],
    [0.0, 1.5*r, 0.0],
    [0.0, 1.5*r, 1.5*r]
], requires_grad=False)

electrons = 4
charge = 0

params, pool = gs_exact(symbols, geometry, electrons, charge, max_iter=200)
import pennylane as qml
from pennylane import numpy as np

active_electrons = 4
active_orbitals = 4
charge = 0
def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2

    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1


def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
    
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
    p_flip = 0.01
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
            qml.BitFlip(p=0.1, wires=1)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #print(FermionicDoubleExcitation(param_array[layer][len(s_wires) + i], wires1=w1, wires2=w2))
       #print(FermionicSingleExcitation(param_array[layer][j], wires=s_w))
        # Insert bit-flip noise on all wires
        #for wire in wires:
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
            qml.BitFlip(p=0.1, wires=1)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
       
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print('exact eigenvalues:', eig)

In [2]:
import pennylane as qml
from pennylane import numpy as np

active_electrons = 4
active_orbitals = 4
charge = 0
def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2

    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1


def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
    print("len(singles) =", len(singles))
    print("len(doubles) =", len(doubles))
    print("len(params) =", len(params))

    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
    p_flip = 0.01
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
            qml.BitFlip(p=0.0, wires=1)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #print(FermionicDoubleExcitation(param_array[layer][len(s_wires) + i], wires1=w1, wires2=w2))
       #print(FermionicSingleExcitation(param_array[layer][j], wires=s_w))
        # Insert bit-flip noise on all wires
        #for wire in wires:
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
            qml.BitFlip(p=0.0, wires=1)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
       
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print('exact eigenvalues:', eig)

len(singles) = 8
len(doubles) = 18
len(params) = 26
exact eigenvalues: [tensor([-1.95195624, -1.89853266, -1.80374151, -1.79717922, -1.77023601,
        -1.70223976, -1.6284237 , -1.43179083, -1.42416975, -1.41790215,
        -1.41162144, -1.35269282, -1.34752373, -1.30037572, -1.28614679,
        -1.23333119, -1.21969271, -1.21849967, -1.12153775, -1.11834081,
        -1.10556785, -1.02453827, -0.91221894, -0.90671381, -0.67388812,
        -0.46464918, -0.44029813], requires_grad=True)]


In [3]:
import pennylane as qml
from pennylane import numpy as np

active_electrons = 4
active_orbitals = 4
charge = 0
def inite(elec, orb):
    list1 = []
    # Single excitations
    list1.append([e for e in range(elec)])
    for x in range(elec):
        config = []
        count = orb - elec
        while count < orb:
            for e in range(elec):
                if x == e:
                    config.append(count + (1 if x % 2 else 0))
                else:
                    config.append(e)
            list1.append(config)
            config = []
            count += 2

    # Double excitations
    for x in range(elec):
        for y in range(x + 1, elec):
            for count1 in range(orb - elec, orb, 2):
                for count2 in range(orb - elec, orb, 2):
                    cont = 1 if (count1 != count2 or (x % 2) != (y % 2)) else 0
                    if (x % 2) == (y % 2) and count2 < count1:
                        cont = 0
                    if cont == 1:
                        config = []
                        for e in range(elec):
                            if x == e:
                                config.append(count1 + (1 if x % 2 else 0))
                            elif y == e:
                                config.append(count2 + (1 if y % 2 else 0))
                            else:
                                config.append(e)
                        list1.append(config)
    return list1


def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)
    #print(qubits)
    #print(wires)
    #num_qubits = len(qubits)
    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
    
    #for t in range(1):
    if shots==0:
        dev = qml.device("default.mixed", wires=qubits)
    else:
        dev = qml.device("default.mixed", wires=qubits,shots=shots)
    #circuit for diagonal part
    p_flip = 0.01
    @qml.qnode(dev)
    def circuit_d(params, occ, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ:
            qml.PauliX(wires=w)
            qml.BitFlip(p=0.1, wires=1)
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
       
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
        #print(FermionicDoubleExcitation(param_array[layer][len(s_wires) + i], wires1=w1, wires2=w2))
       #print(FermionicSingleExcitation(param_array[layer][j], wires=s_w))
        # Insert bit-flip noise on all wires
        #for wire in wires:
        return qml.expval(H)

    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2, wires, s_wires, d_wires, hf_state):
        BasisState(hf_state, wires=wires)
        for w in occ1:
            qml.PauliX(wires=w)
            qml.BitFlip(p=0.1, wires=1)
        first = -1
        for v in occ2:
            if v not in occ1:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
        for v in occ1:
            if v not in occ2:
                if first == -1:
                    first = v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first, v])
       
        param_array = params
        if len(params.shape) == 1:
            param_array = np.expand_dims(params, 0)
        
        for i, (w1, w2) in enumerate(d_wires):
            FermionicDoubleExcitation(param_array[0][len(s_wires) + i], wires1=w1, wires2=w2)
        for j, s_w in enumerate(s_wires):
            FermionicSingleExcitation(param_array[0][j], wires=s_w)
       
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values

eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge, params)
#eigval.append(eig)
print('exact eigenvalues:', eig)

exact eigenvalues: [tensor([-1.79370692+0.j        , -1.68777958+0.j        ,
        -1.60679576+0.j        , -1.60367791+0.j        ,
        -1.58467762-0.05490377j, -1.58467762+0.05490377j,
        -1.49134077+0.j        , -1.48837328+0.j        ,
        -1.42482256+0.j        , -1.39132556+0.j        ,
        -1.30701101+0.j        , -1.28723175+0.j        ,
        -1.2287288 -0.02624617j, -1.2287288 +0.02624617j,
        -1.19825528+0.j        , -1.17480053-0.00351252j,
        -1.17480053+0.00351252j, -1.15413578+0.j        ,
        -1.0810388 +0.j        , -1.06114632+0.j        ,
        -1.0206763 +0.j        , -0.94080093+0.j        ,
        -0.92921134+0.j        , -0.87248684+0.j        ,
        -0.81003241+0.j        , -0.60695467+0.j        ,
        -0.57867632+0.j        ], requires_grad=True)]
